# Interactive Clustering: How analyze an annotation project ?

> #### Table of contents

- [I.](#I): Load project and its data
- [II.](#II): Analyze clustering evolution
- [III.](#III): Explore clustering content
    - [III.A.](#III_A): Choose iteration to analyze
    - [III.B.](#III_B): Select relevant linguistic patterns according to `FMC` analysis.
    - [III.C.](#III_C): Sum up clustering topics with a Large Language Model
    - [III.D.](#III_D): Display clustering with their summaries and their content highlighted according to FMC analysis

> #### Load Python dependencies.

In [1]:
# pip install numpy pandas scipy plotly cognitivefactory_features_maximization_metric openai

In [2]:
# Typing.
from typing import List, Dict, Optional

# File and path management.
import os
import json
import pathlib
import pickle

# Data management matrix.
import pandas as pd
from scipy.sparse import csr_matrix

# Interactive clustering functionnalities.
# from cognitivefactory.interactive_clustering.utils.vectorization import vectorize

# Statistics.
import numpy as np
from sklearn import metrics

# FMC analysis.
from sklearn.feature_extraction.text import TfidfVectorizer
from cognitivefactory.features_maximization_metric.fmc import FeaturesMaximizationMetric

# LLM call.
import time
import openai

# Graph display.
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, HTML

-----
## <a id="I">I.</a> Load project and its data

> #### Set the path to the project data:

In [3]:
PATH_TO_PROJECT_DATA: pathlib.Path = pathlib.Path("./archive-demo-bank-cards-iter-max")

> #### List accessible files in the project.

In [4]:
print(f"Accessible files are: {os.listdir(PATH_TO_PROJECT_DATA)}")

Accessible files are: ['clustering.json', 'constraints.json', 'metadata.json', 'modelization.json', 'sampling.json', 'settings.json', 'status.json', 'texts.json', 'vectors_2D.json', 'vectors_3D.json']


> #### Display project metadata (`project_id`, `project_name`, ...).
> _It needs the files `metadata.json`._

In [5]:
# Load data in a DataFrame/Series.
with open(PATH_TO_PROJECT_DATA / "metadata.json", "r") as metadata_r:
    project_medatata = json.load(metadata_r)
df_metadata: pd.DataFrame = pd.Series(project_medatata)

# Display loaded data.
df_metadata

project_id            archive-demo-bank-cards-iter-max
project_name                         [Demo] Bank cards
creation_timestamp                   1711206849.589537
dtype: object

> #### Display project status (`iteration_id`, `state`, `task`).
> _It needs the files `status.json`._

In [6]:
# Load data in a DataFrame/Series.
with open(PATH_TO_PROJECT_DATA / "status.json", "r") as status_r:
    project_status = json.load(status_r)
df_status: pd.Series = pd.Series(project_status)

# Display loaded data.
df_status

iteration_id               48
state           ITERATION_END
task                     None
dtype: object

> #### Load texts (`text`, `text_preprocessed`, `is_deleted`, ...).
> _It needs the files `texts.json`._

In [7]:
# Load data in a DataFrame/Series.
with open(PATH_TO_PROJECT_DATA / "texts.json", "r") as texts_r:
    project_texts = json.load(texts_r)
df_texts: pd.DataFrame = pd.DataFrame.from_dict(project_texts, orient='index')

# Display loaded data.
print(f">> Total number of texts: '{len(df_texts)}'")
print(f">> Number of deleted texts: '{len(df_texts[df_texts['is_deleted']==True])}'")
df_texts.head()

>> Total number of texts: '499'
>> Number of deleted texts: '0'


,text_original,text,text_preprocessed,is_deleted
0,Comment signaler un vol de carte bleue ?,Comment signaler un vol de carte bleue ?,comment signaler un vol de carte bleue,False
1,Comment signaler une perte de carte de paiement ?,Comment signaler une perte de carte de paiement ?,comment signaler une perte de carte de paiement,False
2,J'ai égaré ma carte de crédit !,J'ai égaré ma carte de crédit !,j ai egare ma carte de credit,False
3,J'ai égaré ma carte de paiement.,J'ai égaré ma carte de paiement.,j ai egare ma carte de paiement,False
4,J'ai perdu ma carte bancaire.,J'ai perdu ma carte bancaire.,j ai perdu ma carte bancaire,False


> #### Load annotated constraints (`data`, `constraint_type`, `to_annotate`, `is_hidden`, `to_fix_conflict`, ...).
> _It needs the files `constraints.json`._

In [8]:
# Load data in a DataFrame/Series.
with open(PATH_TO_PROJECT_DATA / "constraints.json", "r") as constraints_r:
    project_constraints = json.load(constraints_r)
df_constraints: pd.DataFrame = pd.DataFrame.from_dict(project_constraints, orient='index')
df_constraints["data_id_1"] = df_constraints.apply(lambda row: row["data"]["id_1"], axis=1)
df_constraints["data_id_2"] = df_constraints.apply(lambda row: row["data"]["id_2"], axis=1)

# Display loaded data.
print(f">> Total number of constraints: '{len(df_constraints)}'")
print(f">> Number of annotated constraints: '{len(df_constraints[(df_constraints['to_annotate']==False) & (df_constraints['is_hidden']==False)])}'")
print(f">> Number of constraints in conflict: '{len(df_constraints[(df_constraints['to_fix_conflict']==True) & (df_constraints['is_hidden']==False)])}'")
df_constraints.head()

>> Total number of constraints: '2379'
>> Number of annotated constraints: '2379'
>> Number of constraints in conflict: '0'


,data,constraint_type,constraint_type_previous,is_hidden,to_annotate,to_review,to_fix_conflict,comment,date_of_update,iteration_of_sampling,data_id_1,data_id_2
"(400,401)","{'id_1': '400', 'id_2': '401'}",MUST_LINK,[None],False,False,False,False,,1711206980,1,400,401
"(37,38)","{'id_1': '37', 'id_2': '38'}",MUST_LINK,[None],False,False,False,False,,1711206990,1,37,38
"(423,436)","{'id_1': '423', 'id_2': '436'}",MUST_LINK,[None],False,False,False,False,,1711207000,1,423,436
"(229,230)","{'id_1': '229', 'id_2': '230'}",MUST_LINK,[None],False,False,False,False,,1711207010,1,229,230
"(10,11)","{'id_1': '10', 'id_2': '11'}",MUST_LINK,[None],False,False,False,False,,1711207020,1,10,11


> #### Load clustering results.
> _It needs the files `clustering.json`. Result can be `NaN` if text was deleted._

In [9]:
# Load data in a DataFrame.
with open(PATH_TO_PROJECT_DATA / "clustering.json", "r") as clustering_r:
    project_clustering = json.load(clustering_r)

df_clusterings: pd.DataFrame = pd.DataFrame.from_dict(project_clustering)
df_clusterings.fillna(value=-1, inplace=True)  # fill `NaN` values by `-1`.
df_clusterings = df_clusterings.astype(int).astype(str)  # to `int` to remove float, then to `str` to have categorical values.
df_clusterings.sort_index(key=lambda x: x.astype(int), inplace=True)

# Display loaded data.
print(f">> Number of clustering iterations: '{len(project_clustering)}'")
df_clusterings.head()

>> Number of clustering iterations: '49'


,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,1,3,3,3,2,3,4,4,2,1,...,9,9,9,9,9,9,9,9,9,9
1,3,3,3,3,2,3,4,4,2,1,...,9,9,9,9,9,9,9,9,9,9
2,3,1,1,1,1,2,2,2,5,1,...,9,9,9,9,9,9,9,9,9,9
3,3,1,1,1,1,2,2,2,5,1,...,9,9,9,9,9,9,9,9,9,9
4,1,1,1,1,1,2,2,2,5,2,...,9,9,9,9,9,9,9,9,9,9


> #### Load vectorial representation.
> _It needs files `vectors_2D.json`._

In [10]:
# Load data in a DataFrame.
with open(PATH_TO_PROJECT_DATA / "vectors_3D.json", "r") as vectors_3d_r:
    project_vectors_3d = json.load(vectors_3d_r)
df_vectors_3d: pd.DataFrame = pd.DataFrame.from_dict(project_vectors_3d, orient="index")

# Display loaded data.
df_vectors_3d

,x,y,z
0,-42.612156,34.051064,-9.978337
1,-37.619656,38.605362,2.285837
2,11.118752,20.644932,5.942378
3,15.411387,16.566549,6.880816
4,-24.260159,22.722395,20.775551
...,...,...,...
495,39.486561,-42.559635,-6.096722
496,22.253860,-44.178463,-2.633111
497,12.103687,-56.509560,0.959552
498,11.976274,-61.783192,6.321693


-----
## <a id="II">II.</a> Analyze clustering evolution

> ### Display evolution of clustering differences.
> _It is computed by `1 - vmeasure(current_clustering, previous_clustering)`_

In [11]:
# Compute clustering difference score for each iteration.
clustering_differences: Dict[str, float] = {}
for iteration in range(1, int(df_status["iteration_id"])):
    # Get iteration ids.
    current_iteration_id: str = str(iteration)
    previous_iteration_id: str = str(iteration-1)
    # Format clustering results: Get common text ids.
    list_of_common_text_ids: List[str] = [
        text_id
        for text_id in df_clusterings.index
        if (
            df_texts["is_deleted"][text_id] == False
        ) and (
            df_clusterings[previous_iteration_id][text_id] != "-1"
        ) and (
            df_clusterings[current_iteration_id][text_id] != "-1"
        )
    ]
    # Compute scores.
    clustering_differences[current_iteration_id] = 1.0 - metrics.v_measure_score(
        labels_true=df_clusterings[previous_iteration_id][list_of_common_text_ids],
        labels_pred=df_clusterings[current_iteration_id][list_of_common_text_ids],
    )
df_clustering_differences: pd.DataFrame = pd.DataFrame.from_dict(clustering_differences, orient="index", columns=["difference rate"])
df_clustering_differences["iteration"] = df_clustering_differences.index

# Display clustering difference scores.
fig = px.line(
    df_clustering_differences,
    x="iteration",
    y="difference rate",
    markers=True,
    title="<b>Evolution of clustering results differences</b>",
    color_discrete_sequence=["red"],
)
fig.update_layout(yaxis=dict(range=[0.0, 1.0]))
fig

> <details>
>    <summary>Example of clustering difference evolution.</summary>
>    <img src="../figures/analysis-clustering-evolution.png">
> </details>

-----
## <a id="III">III.</a> Explore clustering content

> ### <a id="III_A">III.A.</a> Choose iteration to analyze

> #### Define iteration to analyze (_Defaults to the latest iteration._)

In [12]:
ITERATION_ID: str = str(max(df_clusterings.columns.astype(int)))  # "0"  # str(max(df_clusterings.columns).astype(int))
print(f"Iteration to analyze: {ITERATION_ID}")

Iteration to analyze: 48


> #### Limit clusters to analyze (_Defaults to all clusters without garbage._)

In [13]:
LIST_OF_CLUSTER_IDS: List[str] = sorted([
    cluster_id
    for cluster_id in df_clusterings[ITERATION_ID].unique()
    if cluster_id != "-1"  # do not analyse garbage clusters.
])
print(f"List of clusters to analyze: {LIST_OF_CLUSTER_IDS}")

List of clusters to analyze: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


> #### Limit texts to analyze. (_Defaults to all texts that are not deleted and that are clustered._)

In [14]:
LIST_OF_TEXTS_IDS: pd.Index = (
    (df_texts["is_deleted"]==False)
    & (df_clusterings[ITERATION_ID].isin(LIST_OF_CLUSTER_IDS))
)
print(f"Number of texts to analyze: {sum(LIST_OF_TEXTS_IDS)}")
print(f"List of texts to analyze:\n{LIST_OF_TEXTS_IDS}")

Number of texts to analyze: 499
List of texts to analyze:
0      True
1      True
2      True
3      True
4      True
       ... 
495    True
496    True
497    True
498    True
499    True
Length: 499, dtype: bool


> ### <a id="III_B">III.B.</a> Select relevant linguistic pattern according to `FMC` analysis
> _`FMC` means `Features Maximization Contrast`: it selects the list of linguistic patterns that are the most relevant to represent a cluster and distinguish it from other clusters._

> #### Prepare data for FMC analysis.

In [15]:
# Prepare data and filter deleted data.
df_data_for_fmc: pd.DataFrame = pd.DataFrame()
df_data_for_fmc["text_preprocessed"] = df_texts["text_preprocessed"]
df_data_for_fmc["cluster"] = df_clusterings[ITERATION_ID]
df_data_for_fmc = df_data_for_fmc[LIST_OF_TEXTS_IDS]
df_data_for_fmc["text_with_appropriate_highlighting"] = ""  # Initialize as empty.
df_data_for_fmc["text_with_inappropriate_highlighting"] = ""  # Initialize as empty.
df_data_for_fmc

,text_preprocessed,cluster,text_with_appropriate_highlighting,text_with_inappropriate_highlighting
0,comment signaler un vol de carte bleue,9,,
1,comment signaler une perte de carte de paiement,9,,
2,j ai egare ma carte de credit,9,,
3,j ai egare ma carte de paiement,9,,
4,j ai perdu ma carte bancaire,9,,
...,...,...,...,...
495,puis je mettre en marche le mode sans contact ...,0,,
496,puis je obtenir le sans contact sur ma carte,0,,
497,retirer le paiement sans contact,0,,
498,supprimer le paiement sans contact,0,,


> #### Compute FMC analysis.

In [16]:
# Define vectorizer.
vectorizer = TfidfVectorizer(min_df=0.0, ngram_range=(1, 3), analyzer="word", sublinear_tf=True)
matrix_of_vectors: csr_matrix = vectorizer.fit_transform(df_data_for_fmc["text_preprocessed"])
list_of_possible_vectors_features: List[str] = list(vectorizer.get_feature_names_out())

# Define FMC modelization.
fmc_computer: FeaturesMaximizationMetric = FeaturesMaximizationMetric(
    data_vectors=matrix_of_vectors,
    data_classes=df_data_for_fmc["cluster"],
    list_of_possible_features=list_of_possible_vectors_features,
    amplification_factor=1,
)

# Get most active linguistic patterns accoding to FMC for each cluster.
dict_of_cluster_most_active_linguistic_patterns: Dict[str, List[str]] = {
    cluster_id: [
        linguistic_pattern
        for linguistic_pattern in fmc_computer.get_most_active_features_by_a_classe(
            classe=cluster_id,
            activation_only=True,
            sort_by="fmeasure",  # "fmeasure" or "contrast"
            max_number=50,  # number of linguistic patterns to retain.
        )
        if fmc_computer.get_most_activated_classes_by_a_feature(linguistic_pattern) == [cluster_id]
    ]
    for cluster_id in LIST_OF_CLUSTER_IDS
}

> #### Format clustering content according to FMC analysis.

In [17]:
# Define translation of token in text according to FMC activation.
def format_text_according_to_fmc_analysis(text: str, dict_of_translation: Dict[str, str]) -> str:
    """
        In a text, replace all token according to a translation dictionnary.
        
        Args:
            text (str): The text to translate.
            dict_of_translation (Dict[str, str]): The list of translations. Fro exemple, `card` can be translated in `<b>card</b>`.
        
        Return:
            str: The translated text.
    """
    text_tokenized: List[str] = text.split(" ")
    new_text_tokenized: List[str] = []
    cursor: int = 0
    while cursor < len(text_tokenized):
        # Case of relevant trigram.
        trigram: str = " ".join(text_tokenized[cursor:cursor+3])
        if trigram in dict_of_translation.keys():
            new_text_tokenized.append(dict_of_translation[trigram])
            cursor += 3
            continue
        # Case of relevant bigram.
        bigram: str = " ".join(text_tokenized[cursor:cursor+2])
        if bigram in dict_of_translation.keys():
            new_text_tokenized.append(dict_of_translation[bigram])
            cursor += 2
            continue
        # Case of relevant monogram.
        monogram: str = text_tokenized[cursor]
        if monogram in dict_of_translation.keys():
            new_text_tokenized.append(dict_of_translation[monogram])
            cursor += 1
            continue
        # Default case.
        new_text_tokenized.append(monogram)
        cursor += 1
    return " ".join(new_text_tokenized)

# Highlight relevant linguistic patterns for the current cluster that are present.
df_data_for_fmc["text_with_appropriate_highlighting"] = df_data_for_fmc.apply(
    lambda row: format_text_according_to_fmc_analysis(
        text=row["text_preprocessed"],
        dict_of_translation={
            token: f"<i style='color: #00DD00;'>{token}</i>"
            for token in dict_of_cluster_most_active_linguistic_patterns[row["cluster"]]
        }
    ),
    axis=1,
)

# Highlight relevant linguistic patterns for other clusters that are present.
df_data_for_fmc["text_with_inappropriate_highlighting"] = df_data_for_fmc.apply(
    lambda row: format_text_according_to_fmc_analysis(
        text=row["text_preprocessed"],
        dict_of_translation={
            token: f"<i style='color: #DD0000;'>{token}</i>"
            for other_cluster_id in LIST_OF_CLUSTER_IDS
            for token in dict_of_cluster_most_active_linguistic_patterns[other_cluster_id]
            if row["cluster"] != other_cluster_id
        }
    ),
    axis=1,
)

> ### <a id="III_C">III.C.</a> Sum up clustering topics with a Large Language Model
> _To be able to call a model from OpenAI, you need to register and create a token api key on https://platform.openai.com/account/api-keys, then store it in a file named `credentials.py` near this notebook. The content of this file should look like : `OPENAI_API_TOKEN: str = "..."`._

> #### Prepare data for LLM summary.

In [18]:
# Prepare data and filter deleted data.
df_data_for_llm: pd.DataFrame = pd.DataFrame()
df_data_for_llm["text"] = df_texts["text"]
df_data_for_llm["text_formatted"] = df_data_for_llm.apply(lambda row: f"- {row['text']}", axis=1)
df_data_for_llm["cluster"] = df_clusterings[ITERATION_ID]
df_data_for_llm = df_data_for_llm[LIST_OF_TEXTS_IDS]

# Prepare storage of clustering summary.
dict_of_clustering_summary: Dict[str, Optional[str]] = {
    cluster_id: None
    for cluster_id in LIST_OF_CLUSTER_IDS
}

> #### Define LLM to call.

In [19]:
# Define OpenAI model settings.
import credentials  # The file near this notebook that contains credentials.
openai.api_key = credentials.OPENAI_API_TOKEN  # Given by https://platform.openai.com/account/api-keys.
OPENAI_MODEL: str = "gpt-3.5-turbo"
MAX_RETRY: int = 3  # Retry when timeout.
COOLDOWN_RETRY: int = 21  # # To avoid Error "Rate limit reached" => "default-gpt-3.5-turbo" limited at 3 request per minute.

# Define prompt (adapt according your context).
#PROMPT_SYSTEM: str = """
#Tu es un expert des secteurs banque, assurance et finance.
#Ton objectif est de résumer les thématiques contenues dans les textes suivants.
#Tu répondras en une description concise contenant les informations clées.
#Par exemples: 'Perte et vol de carte bancaire' ou 'Simulation d'un prêt immobilier'.
#"""
PROMPT_SYSTEM: str = """
Tu es un expert du domaine autombile, de la mécanique et de la relation client.
Ton objectif est de résumer les thématiques contenues dans les textes suivants.
Tu répondras en une description concise contenant les informations clées.
Par exemples: 'Réalisation du contrôle technique' ou 'Changement des pneus'.
"""

> #### Call LLM for summary.
> /!\ Set manually the clusters to analyze to avoid additional costs due to LLM call price.

In [20]:
LIST_OF_CLUSTER_IDS_TO_SUM_UP_WITH_AN_LLM: List[str] = []  # LIST_OF_CLUSTER_IDS

In [21]:
# Loop on cluster and sum up their topics.
for cluster_id in LIST_OF_CLUSTER_IDS_TO_SUM_UP_WITH_AN_LLM:

    # Call the LLM to summarize the document (use loop to by-pass timeout).
    nb_of_try: int = 0
    last_error: Exception = None
    while nb_of_try<MAX_RETRY:
        time.sleep(21)  # To avoid "Rate limit reached" => "default-gpt-3.5-turbo" limited at 3 request per minute.
        # Try to get a response from the model.
        try : 
            nb_of_try += 1
            # Get model completion.
            chat_answers = openai.ChatCompletion.create(
                model=OPENAI_MODEL,
                messages=[
                    {
                        "role": "system",
                        "content": f"{PROMPT_SYSTEM}"
                    },
                    {
                        "role": "user",
                        "content": "\n".join(df_data_for_llm[df_data_for_llm["cluster"]==cluster_id]["text_formatted"])
                    }
                ]
            )
            break
        # Catch error.
        except Exception as err:
            last_error = err
            continue
    # If error: continue...
    if nb_of_try==MAX_RETRY:
        print(last_error)
        continue
    # Otherwise: store summary.
    dict_of_clustering_summary[cluster_id] = chat_answers.choices[0].message.content

> ### <a id="III_D">III.D.</a> Display clustering with their summaries and their content highlighted according to FMC analysis

> #### Display clustering content in 3D.

In [24]:
# Prepare data and filter deleted data.
df_clustering_3d: pd.DataFrame = df_vectors_3d.copy()
df_clustering_3d["cluster"] = df_clusterings[ITERATION_ID]
df_clustering_3d["text"] = df_texts["text"]
df_clustering_3d = df_clustering_3d[LIST_OF_TEXTS_IDS]

# Display clusters.
fig = px.scatter_3d(
    df_clustering_3d, 
    x="x",
    y="y",
    z="z",
    color="cluster",
    hover_name="text",
    title=f"<b>Clustering result at iteration '{ITERATION_ID}'.</b>",
    width=800,
    height=800,
)
fig.update_layout(
    xaxis=dict(range=[min(df_clustering_3d["x"])-1, max(df_clustering_3d["x"])+1]),
    yaxis=dict(range=[min(df_clustering_3d["y"])-1, max(df_clustering_3d["y"])+1]),
)
fig

> <details>
>    <summary>Example of clustering 3D display.</summary>
>    <img src="../figures/analysis-clustering-3d.png">
> </details>

> #### Display clustering content analysis.

In [23]:
# Display summary of clusters.
for cluster_id in LIST_OF_CLUSTER_IDS:
	display(HTML(f"""
		<hr>
        <details>
			<summary style="font-weight: bold; font-size: 1.2em;">Display summary of cluster '{cluster_id}':  Length='{len(df_data_for_fmc[df_data_for_fmc["cluster"]==cluster_id])}'  ;  Summary='<i style='color:orange;'>{dict_of_clustering_summary[cluster_id]}</i>'</summary>
			<p style='text-align: justify;'>
				<b>Linguistic patterns:</b>
				<i>{dict_of_cluster_most_active_linguistic_patterns[cluster_id]}</i>
			</p>
			<table>
				<thead>
					<tr>
						<th>Text ID</th>
						<th>Text with appropriate highlighting</th>
						<th>Text with inappropriate highlighting</th>
					</tr>
				</thead>
				<tbody>
					{
						''.join([
							f'''
							<tr>
								<th>{text_id}</th>
								<td>{row["text_with_appropriate_highlighting"]}</td>
								<td>{row["text_with_inappropriate_highlighting"]}</td>
							</tr>
							'''
							for text_id, row in df_data_for_fmc.iterrows()
							if row['cluster'] == cluster_id
						])
					}
				</tbody>
			</table>
		</details>
	"""))

Text ID,Text with appropriate highlighting,Text with inappropriate highlighting
450,activer le moyen de paiement nfc sur ma carte gold,activer le moyen de paiement nfc sur ma carte gold
451,activer l option nfc sur ma mastercard,activer l option nfc sur ma mastercard
452,activez moi le sans contact s il vous plait,activez moi le sans contact s il vous plait
453,activiation paiement sans contact,activiation paiement sans contact
454,bloquer le mode de paiement nfc,bloquer le mode de paiement nfc
455,comment activer l option paiement sans contact,comment activer l option paiement sans contact
456,comment desactiver le sans contact sur ma mastercard,comment desactiver le sans contact sur ma mastercard
457,comment enlever le mode nfc sur une carte bancaire,comment enlever le mode nfc sur une carte bancaire
458,comment mettre en marche le mode nfc sur ma carte bleue,comment mettre en marche le mode nfc sur ma carte bleue
459,comment savoir si le paiement nfc est active sur ma carte,comment savoir si le paiement nfc est active sur ma carte


Text ID,Text with appropriate highlighting,Text with inappropriate highlighting
350,ai je la possibilite d avoir un decouvert,ai je la possibilite d avoir un decouvert
351,ai je le droit d avoir un decouvert bancaire,ai je le droit d avoir un decouvert bancaire
352,ai je l obligation d avoir un compte bancaire crediteur,ai je l obligation d avoir un compte bancaire crediteur
353,ai je une autorisation de decouvert,ai je une autorisation de decouvert
354,comment augmenter sa limite de decouvert,comment augmenter sa limite de decouvert
355,comment avoir un plus grand decouvert,comment avoir un plus grand decouvert
356,comment beneficier d un decouvert,comment beneficier d un decouvert
357,comment demander une autorisation de decouvert,comment demander une autorisation de decouvert
358,comment gerer la possibilite de decouvert,comment gerer la possibilite de decouvert
359,comment modifier sa limite de decouvert,comment modifier sa limite de decouvert


Text ID,Text with appropriate highlighting,Text with inappropriate highlighting
300,activer les achats avec un numero virtuel,activer les achats avec un numero virtuel
301,activer les numeros de carte virtuelle,activer les numeros de carte virtuelle
302,ai je le droit de creer des numeros de carte pour des paiements online,ai je le droit de creer des numeros de carte pour des paiements online
303,ai je le droit de creer un numero virtuel,ai je le droit de creer un numero virtuel
304,bonjour comment gerer ses numeros de carte virtuelle,bonjour comment gerer ses numeros de carte virtuelle
305,comment acheter en ligne avec un numero de carte bancaire virtuelle,comment acheter en ligne avec un numero de carte bancaire virtuelle
306,comment activer les paiements virtuels,comment activer les paiements virtuels
307,comment annuler la creation d un numero de carte virtuelle,comment annuler la creation d un numero de carte virtuelle
308,comment consulter ses numeros de carte virtuelle,comment consulter ses numeros de carte virtuelle
309,comment faire pour creer une carte de paiements virtuelle,comment faire pour creer une carte de paiements virtuelle


Text ID,Text with appropriate highlighting,Text with inappropriate highlighting
200,ai je une extension de garantie avec ma carte gold pour un achat d eletromenager,ai je une extension de garantie avec ma carte gold pour un achat d eletromenager
201,assurance comprises avec ma carte bancaire pour un voyage a l etranger,assurance comprises avec ma carte bancaire pour un voyage a l etranger
202,carte de credit et garanties d electromenager,carte de credit et garanties d electromenager
203,comment faire valoir l assurance de ma mastercard,comment faire valoir l assurance de ma mastercard
204,comment fonctionne l assurance de ma carte,comment fonctionne l assurance de ma carte
205,comment suis je couvert avec ma carte en cas d accident,comment suis je couvert avec ma carte en cas d accident
206,comment utiliser l assurance de ma carte de credit,comment utiliser l assurance de ma carte de credit
207,consulter les droits d assurance de ma carte,consulter les droits d assurance de ma carte
208,couverture carte bancaire pour la reparation d une television,couverture carte bancaire pour la reparation d une television
209,de quelles couvertures d assurance je dispose avec ma carte gold,de quelles couvertures d assurance je dispose avec ma carte gold


Text ID,Text with appropriate highlighting,Text with inappropriate highlighting
250,apres plusieurs mauvais codes ma carte a ete bloquee,apres plusieurs mauvais codes ma carte a ete bloquee
251,bonjour j ai bloque ma carte et je voudrai la debloquer merci d avance,bonjour j ai bloque ma carte et je voudrai la debloquer merci d avance
252,bonjour pouvez vous debloquer ma carte merci,bonjour pouvez vous debloquer ma carte merci
253,bonjour serait il possible de debloquer ma carte s il vous plait,bonjour serait il possible de debloquer ma carte s il vous plait
254,comment annuler le blocage de ma carte,comment annuler le blocage de ma carte
255,comment debloquer ma mastercard,comment debloquer ma mastercard
256,comment debloquer rapidement une carte de credit,comment debloquer rapidement une carte de credit
257,comment debloquer sa carte bleue,comment debloquer sa carte bleue
258,comment deverrouiller sa carte,comment deverrouiller sa carte
259,comment deverrouiller sa carte bancaire,comment deverrouiller sa carte bancaire


Text ID,Text with appropriate highlighting,Text with inappropriate highlighting
150,a combien s eleve le solde de mon compte,a combien s eleve le solde de mon compte
151,bonjour comment prendre connaissance du solde de mon compte courant,bonjour comment prendre connaissance du solde de mon compte courant
152,combien ai je sur mon compte,combien ai je sur mon compte
153,combien ai je sur mon compte courant,combien ai je sur mon compte courant
154,combien d argent me reste t il sur mon compte,combien d argent me reste t il sur mon compte
155,combien me reste t il d argent,combien me reste t il d argent
156,combien me reste t il sur mon compte,combien me reste t il sur mon compte
157,comment connaitre la situation de mon compte bancaire,comment connaitre la situation de mon compte bancaire
158,comment connaitre l etat de mes comptes,comment connaitre l etat de mes comptes
159,comment consulter mon compte courant,comment consulter mon compte courant


Text ID,Text with appropriate highlighting,Text with inappropriate highlighting
400,augmentation du plafond de ma carte,augmentation du plafond de ma carte
401,augmentation du plafond de ma mastercard,augmentation du plafond de ma mastercard
402,augmentation temporaire du plafond,augmentation temporaire du plafond
403,augmenter la plafond de ma carte visa,augmenter la plafond de ma carte visa
404,augmenter le plafond de ma carte bancaire,augmenter le plafond de ma carte bancaire
405,augmenter mon plafond de paiement a l etranger,augmenter mon plafond de paiement a l etranger
406,augmenter sa capacite de paiement temporairement,augmenter sa capacite de paiement temporairement
407,augmenter temporairement ma limite de retrait,augmenter temporairement ma limite de retrait
408,besoin de modifier le plafond de retrait de ma carte bleue,besoin de modifier le plafond de retrait de ma carte bleue
409,bonjour comment puis je augmenter le plafond de ma carte bancaire,bonjour comment puis je augmenter le plafond de ma carte bancaire


Text ID,Text with appropriate highlighting,Text with inappropriate highlighting
100,changer de carte bancaire,changer de carte bancaire
101,changer de carte de credit,changer de carte de credit
102,changer de carte de paiement pour une carte sans frais,changer de carte de paiement pour une carte sans frais
103,choisir une nouvelle carte bancaire,choisir une nouvelle carte bancaire
104,commander une carte a paiement differe,commander une carte a paiement differe
105,commander une carte avance sante,commander une carte avance sante
106,commander une carte bancaire,commander une carte bancaire
107,commander une carte visa,commander une carte visa
108,commander une nouvelle carte bleue,commander une nouvelle carte bleue
109,comment avoir une carte gold,comment avoir une carte gold


Text ID,Text with appropriate highlighting,Text with inappropriate highlighting
50,carte avalee,carte avalee
51,carte bancaire avalee,carte bancaire avalee
52,carte bancaire retenue au distributeur,carte bancaire retenue au distributeur
53,carte bleue avalee,carte bleue avalee
54,carte de paiement avalee,carte de paiement avalee
55,comment faire pour chercher une carte bancaire avalee,comment faire pour chercher une carte bancaire avalee
56,comment recuperer une carte avalee,comment recuperer une carte avalee
57,comment recuperer une mastercard retenue par le distributeur,comment recuperer une mastercard retenue par le distributeur
58,en cherchant de l argent le gab a avale ma carte,en cherchant de l argent le gab a avale ma carte
59,en retirant de l argent le distributeur a garde ma carte,en retirant de l argent le distributeur a garde ma carte


Text ID,Text with appropriate highlighting,Text with inappropriate highlighting
0,comment signaler un vol de carte bleue,comment signaler un vol de carte bleue
1,comment signaler une perte de carte de paiement,comment signaler une perte de carte de paiement
2,j ai egare ma carte de credit,j ai egare ma carte de credit
3,j ai egare ma carte de paiement,j ai egare ma carte de paiement
4,j ai perdu ma carte bancaire,j ai perdu ma carte bancaire
5,j ai perdu ma carte bleue,j ai perdu ma carte bleue
6,j aimerai declarer un vol de carte,j aimerai declarer un vol de carte
7,je me suis fait voler ma carte bancaire,je me suis fait voler ma carte bancaire
8,je me suis fait voler ma carte bleue,je me suis fait voler ma carte bleue
9,je n ai pas retrouve ma carte bancaire depuis plusieurs jours,je n ai pas retrouve ma carte bancaire depuis plusieurs jours
